In [2]:
# importing libs
import seaborn as sns
import tqdm
from tqdm import tqdm
import sklearn
from sklearn.metrics import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
val_files = os.listdir('val/')
id = []
result = []

In [5]:
for i in range(0,len(val_files),1):
    a = val_files[i].replace('.jpeg','')
    id.append(a)
    if a.find('NORMAL'):
        result.append(1)
    else:
        result.append(0)

In [6]:
normal_xray = len(os.listdir('train/NORMAL'))
sick_xray = len(os.listdir('train/PNEUMONIA'))
number_train = normal_xray + sick_xray

In [7]:
normal_xray_test = len(os.listdir('test/NORMAL'))
sick_xray_test = len(os.listdir('test/PNEUMONIA'))
number_test = normal_xray_test + sick_xray_test